In [1]:
#https://sourceforge.net/projects/swig/files/swigwin/swigwin-4.0.2/swigwin-4.0.2.zip/download?use_mirror=ixpeering

#https://gymnasium.farama.org/ 

# 1. Import Dependencies

In [2]:
import gymnasium as gym
from gymnasium import spaces
from gymnasium import Env

import numpy as np
import random
import os

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

# Controls the keys
from pynput.keyboard import key, Controller

# 3. Building an Environment

# 6. Save Model

In [3]:
# Create custom environment
class TheBindingAI(gym.Env):
  metadata = {'render.modes': ['human']}
  # Max number of objects allowed
  MAX_OBJECTS = 10
  # Each object has 3 features, for example.
  OBJECT_FEATURES = 3 



  def __init__(self):
    # make bigger, same and less
    self.action_space = spaces.Discrete(7) #0 = w, 1=a, 2=s, 3=d, 4='space', 5=arrowup, 6=arrowleft, 7=arrowdown, 8=arrowright
    self.observation_space = spaces.Box(
        low=-np.inf, 
        high=np.inf, 
        shape=(self.MAX_OBJECTSMAX_OBJECTS, self.OBJECT_FEATURES), 
        dtype=np.float32
      ) #Hier input der einzelnen Objecte im Raum
    self.state = 880
    self.recentstate = 880 
    self.training_length = 120



  # Eine Beispiel-Funktion zur Erstellung einer Beobachtung mit einer variablen Anzahl von Objekten
def get_observation(self, num_objects):
    # Sicherstellen, dass die Anzahl der Objekte die festgelegte Obergrenze nicht überschreitet
    assert num_objects <= self.MAX_OBJECTS
    
    # Erstellen eines zufälligen Arrays (normalverteilte Zahlen) für die angegebene Anzahl von Objekten.
    # Jedes Objekt hat "OBJECT_FEATURES" Merkmale (Spalten).
    obs = np.random.randn(num_objects, self.OBJECT_FEATURES)
    
    # Erstellen eines Arrays der festen Größe (MAX_OBJECTS x OBJECT_FEATURES), gefüllt mit Nullen.
    # Dieses Array wird später mit den tatsächlichen Beobachtungen aufgefüllt.
    padded_obs = np.zeros((self.MAX_OBJECTS, self.OBJECT_FEATURES), dtype=np.float32)
    
    # Kopieren der zufällig generierten Beobachtungen (obs) in das gepolsterte Array (padded_obs).
    # Nur die ersten "num_objects" Zeilen des gepolsterten Arrays werden mit Werten gefüllt.
    padded_obs[:num_objects] = obs
    
    # Rückgabe des vollständig gepolsterten Arrays, das jetzt die zufälligen Beobachtungen enthält.
    # Nicht genutzte Zeilen bleiben als Nullwerte bestehen.
    return padded_obs



  def step(self, action):
    # Apply temp adj
    self.state += # IngameScore

    #decrease shower time/Periode time
    self.traininglength -= 1

    #calculate reward ==> Score
    if self.state + 10 >= self.recentstate:
      reward = 1

    elif self.state - 10 <= self.recentstate:
      reward = -1
    
    # stops the learning episode
    if self.training_length <= 0:
      # episode ends naturally
      terminated = True
    else: 
       terminated = False

    #epiode ends prematurely // Time limit
    truncated = False
    info = {}
    return self.state, reward, terminated, truncated, info
  
  
  def reset(self, seed=None, options=None):
    # Seed the environment if a seed is provided
    if seed is not None:
        np.random.seed(seed)
        random.seed(seed)


    # Reset the environment state
    self.state = 880
    self.training_length = 120

    # Return observation and optional info
    return self.state, {}
  
  def render(self, mode='human'):
    #none
    pass
  
  def close (self):
    pass


IndentationError: unindent does not match any outer indentation level (<string>, line 48)

In [4]:
env=TheBindingAI()

In [5]:
env.observation_space.sample()

array([9.0777445], dtype=float32)

In [6]:
env.reset()

(array([39.]), {})

# 4. Test Environment

In [7]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()

        obs, reward, terminated, truncated, info = env.step(action)
        
        done = terminated or truncated

        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:-60
Episode:2 Score:-58
Episode:3 Score:-60
Episode:4 Score:-32
Episode:5 Score:-54


In [8]:
env.close()

# 5. Train Model

In [9]:
log_path = os.path.join('Training', 'Logs')
model = PPO("MlpPolicy", env, learning_rate=(0.0005), verbose=1, tensorboard_log=log_path)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [10]:
model.learn(total_timesteps=200000)

Logging to Training\Logs\PPO_1
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -38.6    |
| time/              |          |
|    fps             | 807      |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | -41.3       |
| time/                   |             |
|    fps                  | 580         |
|    iterations           | 2           |
|    time_elapsed         | 7           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.004011985 |
|    clip_fraction        | 0.0422      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.09       |
|    explained_variance   | -0.000493   |

# 6. Save Model

In [11]:
model_path = os.path.join('Training', 'Saved Models', 'TheBindingOfIsaac_PPO')

In [12]:
model.save(model_path)

In [13]:
del model

In [14]:
model = PPO.load(model_path, env)

PermissionError: [Errno 13] Permission denied: 'Training\\Saved Models\\TheBindingOfIsaac_PPO'

In [ ]:
evaluate_policy(model, env, n_eval_episodes=200, render=False)

(8.74, 59.10103552392292)